In [1]:
# Denpendances
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
# Generating gain and loss percentage data
def clip_dataframe(df, p):
    # Batching for test data
    batches = len(df) // p
    max_length = batches * p
    return df.iloc[-max_length:].copy()

def simple_gain_loss(data, period):
    # Get gain and loss values base on start of period and
    # end of period prices
    ndf = clip_dataframe(data, period)
    # Base period prices
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    # Shifting one day to avoid the end period price is the
    # base price price
    shifted = ndf.shift(-1).fillna(method='ffill')
    target = np.array(shifted['Close']).reshape(-1, period)
    # Calculate gain and loss price array
    gain = target.max(1) / base - 1
    loss = target.min(1) / base - 1
    return gain.mean(), loss.mean()

def average_daily_fluctuation(data, minute=30):
    # Calculate an average daily fluctuation percentage value
    hl = data['High'] / data['Low'] - 1
    minutes = 7.5 * (60 / minute)
    return hl.mean() / minutes

def get_values(data, period, minute=30):
    # Getting values of sell limit, stop loss and fluctuation percentage
    sell_limit, stop_loss = simple_gain_loss(data, period)
    fluct = average_daily_fluctuation(data, minute)
    return sell_limit, stop_loss, fluct

In [3]:
# Static back test
def not_so_simple_gain_loss(data, period):
    ndf = clip_dataframe(data, period)
    # Base
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    # Expanding dimension
    base = np.expand_dims(base, 1)
    ndf = clip_dataframe(data, period)
    # base
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    base = np.expand_dims(base, 1)
    # shifted
    shift = ndf.shift(-1).fillna(method='ffill')
    # High
    high = np.array(shift['High']).reshape(-1, period)
    # Low
    low = np.array(shift['Low']).reshape(-1, period)
    # Open
    Open = np.array(shift['Open']).reshape(-1, period)
    # Close
    close = np.array(shift['Close']).reshape(-1, period)
    # Gaining percentage
    high_gains = high / base - 1
    low_gains = low / base - 1
    close_gains = close / base - 1
    open_gains = Open / base - 1
    return open_gains, high_gains, low_gains, close_gains

def calculate_gains(data, period, sell_limit, stop_loss, fluct):
    open_gains, high_gains, low_gains, close_gains = not_so_simple_gain_loss(data, period)
    # Getting gained percentage 
    length = len(open_gains)
    arr = []
    for i in range(length):
        sold = False
        for p in range(period):
            if open_gains[i, p] < stop_loss:
                arr.append(open_gains[i, p])
                sold = True
                break
            elif high_gains[i, p] > sell_limit:
                arr.append(high_gains[i, p] - fluct)
                sold = True
                break
            elif low_gains[i, p] < stop_loss:
                arr.append(stop_loss)
                sold = True
                break
        if not sold:
            arr.append(close_gains[i, -1])
    return np.array(arr)

def calculate_capital_gain(gains, capital=1):
    original_capital = capital
    for gain in gains:
        capital *= 1 + gain
    return capital / original_capital - 1

In [16]:
stock = 'MSFT'
d1 = DataLoader(stock).get_data('2019-01-01', '2020-12-31')
d2 = DataLoader(stock).get_data('2021-01-01', '2021-12-31')

In [17]:
print(' Period | Sell Limit % |  Stop Loss % | Capital Gain % ')
for i in range(3, 21):
    sell_limit, stop_loss, fluct = get_values(d1, i)
    gains = calculate_gains(d2, i, sell_limit, stop_loss, fluct)
    capital_gain_percentage = calculate_capital_gain(gains)
    print(' {:6d} | {:>12.2f} | {:>12.2f} | {:>14.2f} '.format(
        i, sell_limit * 100, stop_loss * 100,
        capital_gain_percentage * 100))


 Period | Sell Limit % |  Stop Loss % | Capital Gain % 
      3 |         1.33 |        -0.91 |          37.37 
      4 |         1.76 |        -1.09 |          47.60 
      5 |         2.47 |        -1.23 |          26.28 
      6 |         2.34 |        -1.60 |          56.45 
      7 |         3.35 |        -1.07 |          29.07 
      8 |         3.28 |        -1.67 |          21.66 
      9 |         3.90 |        -1.52 |          14.71 
     10 |         3.90 |        -1.54 |          15.79 
     11 |         4.08 |        -2.18 |          56.35 
     12 |         3.92 |        -2.50 |          35.99 
     13 |         4.09 |        -2.52 |          17.39 
     14 |         5.00 |        -2.15 |          28.58 
     15 |         4.79 |        -2.84 |          30.99 
     16 |         5.45 |        -2.53 |          15.49 
     17 |         4.97 |        -3.23 |          31.25 
     18 |         6.30 |        -2.15 |          22.38 
     19 |         5.26 |        -2.91 |         